In [2]:
import numpy as np
import pandas as pd
import scipy as sp
import pylab as pl
import scipy.cluster.hierarchy as hier
import scipy.spatial.distance as dist
import matplotlib as plt
%matplotlib inline

In [8]:
gene = pd.read_excel('gene_expression_table4.xlsx')
#print(gene)
#datamatrix=[]
del gene['Unnamed: 9'] # deleted column without any content
#append each column except the first column which is the gene ID
just_expression=gene.iloc[0:373,3:14]

In [10]:
from sklearn.linear_model import Lasso
def lasso_regression(gene, just_expression, alpha, models_to_plot={}):
    #Fit the model
    lassoreg = Lasso(alpha=alpha,normalize=True, max_iter=1e5)
    lassoreg.fit(data[predictors],data['y'])
    y_pred = lassoreg.predict(data[predictors])
    
    #Check if a plot is to be made for the entered alpha
    if alpha in models_to_plot:
        plt.subplot(models_to_plot[alpha])
        plt.tight_layout()
        plt.plot(data['x'],y_pred)
        plt.plot(data['x'],data['y'],'.')
        plt.title('Plot for alpha: %.3g'%alpha)
    
    #Return the result in pre-defined format
    rss = sum((y_pred-data['y'])**2)
    ret = [rss]
    ret.extend([lassoreg.intercept_])
    ret.extend(lassoreg.coef_)
    return ret

SyntaxError: invalid syntax (<ipython-input-10-cfba141d0abe>, line 4)